#### Important Links (for attribute and database reference):

https://datasus.saude.gov.br/mortalidade-desde-1996-pela-cid-10

https://renastonline.ensp.fiocruz.br/sites/default/files/wiki/dicionario-sim.pdf

https://diaad.s3.sa-east-1.amazonaws.com/sim/Mortalidade_Geral+-+Estrutura.pdf

https://svs.aids.gov.br/download/Dicionario_de_Dados_SIM_tabela_DO.pdf

http://tabnet.datasus.gov.br/cgi/sim/Mortalidade_Geral_1996_2012.pdf

http://tabnet.datasus.gov.br/cgi/sim/Consolida_Sim_2011.pdf

ICD10 suicide codes used by DATASUS:

Códigos de suicídio usados pelo DATASUS do CID10:
http://www2.datasus.gov.br/cid10/V2008/WebHelp/v01_y98.htm#Cap20Nota01

#### DATASUS DATA:
```
ftp.datasus.gov.br
    /dissemin/publicos/SIM/CID10/DORES
    /dissemin/publicos/SIM/CID10/DOCS
```
Usual year update date:

![image.jpg](../utils/infos/attributes-desc/imgs/datasus_year_update_date.jpg)

In [675]:
# Importações
from IPython.display import display

import pandas as pd

import numpy as np
import datetime

import matplotlib.pyplot as plt

# if automatic converters are necessary
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Prevent infinite warnings
import warnings
warnings.filterwarnings('ignore')

In [676]:
# OS and File imports
import os
import sys

import importlib

from zipfile import ZipFile
from io import BytesIO

# List of directories to add to sys.path
directories_to_add = [
    os.path.abspath(os.path.join('..')),
    os.path.abspath(os.path.join('../utils')),
    os.path.abspath(os.path.join('../utils', 'functions')),
]

for directory in directories_to_add:
    if directory not in sys.path:
        sys.path.insert(0, directory)
print(sys.path)

# Making sure pycache is remade on kernel run since files are tiny
import color_palettes
importlib.reload(color_palettes)
import attributes_values_table_generator
importlib.reload(attributes_values_table_generator)
import preprocessing
importlib.reload(preprocessing)

from color_palettes import my_blue, my_orange
from attributes_values_table_generator import generate_attributes_values_table, get_values_table_columns, append_to_attributes_values_table, update_attribute_values, get_attributes_values_table
from preprocessing import initial_df_dtypes, initial_parse_dates, float_to_int

['d:\\.MESTRADO\\.Orientacao\\BrSuicides-dataset\\utils\\functions', 'd:\\.MESTRADO\\.Orientacao\\BrSuicides-dataset\\utils', 'd:\\.MESTRADO\\.Orientacao\\BrSuicides-dataset', 'c:\\Python312\\python312.zip', 'c:\\Python312\\DLLs', 'c:\\Python312\\Lib', 'c:\\Python312', '', 'C:\\Users\\pp0l0\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\pp0l0\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32', 'C:\\Users\\pp0l0\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\pp0l0\\AppData\\Roaming\\Python\\Python312\\site-packages\\Pythonwin', 'c:\\Python312\\Lib\\site-packages']


In [677]:
# ---------------- USER SET FOLDER PATH ----------------
user_dir_path = ''

root_dir = '..'

final_dataset_dir = '/BrSuicides/'
final_dataset_name = 'BrSuicides'

csv_dir = '/data_storage/'
# csv_dir_datasus = csv_dir + '0_datasus_csvs/'
# csv_dir_dirty = csv_dir + '1_dirty/'
csv_dir_filtered = csv_dir + '2_filtered/'
csv_dir_final = final_dataset_dir + '/'

csv_data_dir = os.path.dirname(root_dir + csv_dir) + '/'
print(csv_data_dir)

# datasus_data_dir = os.path.dirname(root_dir + csv_dir_datasus) + '/'
# print(datasus_data_dir)
# csv_data_dir_dirty = os.path.dirname(root_dir + csv_dir_dirty) + '/'
# print(csv_data_dir_dirty)
csv_data_dir_filtered = os.path.dirname(root_dir + csv_dir_filtered) + '/'
print(csv_data_dir_filtered)
csv_data_dir_final = os.path.dirname(root_dir + csv_dir_final) + '/'
print(csv_data_dir_final)

../data_storage/
../data_storage/2_filtered/
../BrSuicides/


In [678]:
# List all files in the csvs directory
all_files = os.listdir(csv_data_dir_filtered)

# Filter for files that end with .zip extension
csv_files = [file for file in all_files if file.endswith('.csv')]
print('CSVs: ', '\n', csv_files)

# Filter for files that end with .zip extension
csv_file_name = csv_files[0]
print('CSV file name: ', '\n', csv_file_name)

years_interval = csv_file_name.rsplit('-', 1)[1][0:-4].split('_')
print('Years interval: ', '\n' , years_interval)

# Years that will be downloaded, [1996,2023) interval
years = [x for x in range(int(years_interval[0]), (int(years_interval[-1]) + 1))]

print('Years:', '\n', years)

CSVs:  
 ['suicidios-brazil-1996_2022.csv']
CSV file name:  
 suicidios-brazil-1996_2022.csv
Years interval:  
 ['1996', '2022']
Years: 
 [1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


### Dataframe:

### Como os dados são coletados e sua granularidade:

As Secretarias de Saúde coletam as Declarações de Óbitos dos cartórios e entram, no SIM, as informações nelas contidas. Uma das informações primordiais é a causa básica de óbito, a qual é codificada a partir do declarado pelo médico atestante, segundo regras estabelecidas pela Organização Mundial de Saúde. A partir de 1996, as declarações de óbito passaram a ser codificadas utilizando-se a 10ª Revisão da Classificação Internacional de Doenças - CID-10. Até então, era utilizada a 9ª Revisão - CID-9. Devido às diferenças entre as revisões, não foi possível gerar uma lista que as compatibilizasse.

De uma maneira geral, as Secretarias Municipais e Estaduais de Saúde enviam periodicamente a sua Base de Dados para o Ministério da Saúde. Este só pode considerar a Base Nacional completa quando todas as UF enviaram seus dados. A partir daí, é feita a consolidação, inclusive com a redistribuição dos óbitos pelo local de residência, a qual é a forma tradicional de apresentar os dados de Mortalidade. Eventualmente, são feitas algumas correções nas informações, em conjunto com as Secretarias Municipais e Estaduais, principalmente quanto ao cruzamento de causa de óbito por sexo e idade.

### How the data is collected and their granularity:

The Health Departments collect Death Certificates from the registry offices and enter the information into the Mortality Information System (SIM). One of the primary pieces of information is the underlying cause of death, which is coded based on what the certifying physician declares, following rules established by the World Health Organization. Since 1996, death certificates have been coded using the 10th Revision of the International Classification of Diseases (ICD-10). Prior to that, the 9th Revision (ICD-9) was used. Due to differences between the revisions, it was not possible to generate a list that made them compatible.

Generally, Municipal and State Health Departments periodically send their databases to the Ministry of Health. The National Database can only be considered complete when all the states have submitted their data. From there, consolidation is done, including the redistribution of deaths by place of residence, which is the traditional way to present Mortality data. Occasionally, some corrections are made to the information, in conjunction with the Municipal and State Health Departments, especially concerning the cross-referencing of cause of death by sex and age.

Referência / Reference:
http://tabnet.datasus.gov.br/cgi/sim/Mortalidade_Geral_1996_2012.pdf


In [679]:
dataframe = pd.read_csv(csv_data_dir_filtered + csv_file_name, encoding='utf-8', dtype=initial_df_dtypes, parse_dates=initial_parse_dates)
display(dataframe.info())
display(dataframe)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267976 entries, 0 to 267975
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   ESTADO      267976 non-null  object        
 1   ANO         267976 non-null  int64         
 2   DTOBITO     267976 non-null  datetime64[ns]
 3   NATURAL     222906 non-null  object        
 4   DTNASC      266011 non-null  object        
 5   IDADE       267817 non-null  object        
 6   SEXO        267972 non-null  object        
 7   RACACOR     246441 non-null  object        
 8   ESTCIV      259609 non-null  object        
 9   ESC         226836 non-null  object        
 10  OCUP        217938 non-null  object        
 11  CODMUNRES   267976 non-null  int64         
 12  LOCOCOR     267572 non-null  object        
 13  CODMUNOCOR  267976 non-null  int64         
 14  CAUSABAS    267976 non-null  object        
 15  ESC2010     128028 non-null  object        
 16  ES

None

,ESTADO,ANO,DTOBITO,NATURAL,DTNASC,IDADE,SEXO,RACACOR,ESTCIV,ESC,OCUP,CODMUNRES,LOCOCOR,CODMUNOCOR,CAUSABAS,ESC2010,ESCFALAGR1
0,PR,1996,1996-10-24,841,5081963,433,1,NaN,1,1,61200,4119251,5,4119251,X709,NaN,NaN
1,PR,1996,1996-02-04,152,21091920,475,1,NaN,2,1,62100,4101903,3,4101903,X680,NaN,NaN
2,PR,1996,1996-01-17,841,28081961,434,1,NaN,1,NaN,62100,4100103,1,4100103,X680,NaN,NaN
3,PR,1996,1996-02-07,841,24091977,418,1,NaN,1,1,62100,4123204,1,4126009,X680,NaN,NaN
4,PR,1996,1996-02-09,841,30081956,439,1,NaN,5,NaN,62100,4123907,3,4123907,X700,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267971,MT,2022,2022-09-17,851,28081982,440,1,4,5,2,622005,510395,3,510395,X730,1,01
267972,MT,2022,2022-08-21,815,11091998,423,1,4,1,4,715210,510340,5,510340,X708,2,04
267973,MT,2022,2022-10-14,851,23061993,429,1,1,1,4,914405,510760,3,510760,X700,3,06
267974,MT,2022,2022-10-09,851,12061981,441,1,4,1,5,354705,510760,3,510760,X700,4,07


In [680]:
dataframe_columns = list(dataframe.columns)

print('Dataframe columns:\n', dataframe_columns)
print('\nNº of columns in dataframe: ', len(dataframe_columns))

Dataframe columns:
 ['ESTADO', 'ANO', 'DTOBITO', 'NATURAL', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'ESTCIV', 'ESC', 'OCUP', 'CODMUNRES', 'LOCOCOR', 'CODMUNOCOR', 'CAUSABAS', 'ESC2010', 'ESCFALAGR1']

Nº of columns in dataframe:  17


### Dicionário de conversões de valores para os atributos:

De acordo com a referência, estes são todos os atributos que necessitam de uma tabela de conversão de valores nominais para descritivos, com as informações relevantes de nome do atributo, sua descrição, e ano inicial de inclusão:

Os atributos comentados são atributos filtrados futuramente, removidos agora para reduzir o tamanho do dicionário.

### Dictionary of Value Conversions for Attributes:
According to the reference, these are all the attributes that require a conversion table from nominal to descriptive values, with relevant information about the attribute name, its description, and the starting year of inclusion:

The commented attributes are those that will be filtered out in the future, removed now to reduce the size of the dictionary.

Referência / Reference:
https://diaad.s3.sa-east-1.amazonaws.com/sim/Mortalidade_Geral+-+Estrutura.pdf

In [681]:
from utils.functions.conversion_dict import get_conversion_dict

conversion_dict = get_conversion_dict()

print('Possible nº of categorical data needing conversion: ', len(conversion_dict.keys()))

Possible nº of categorical data needing conversion:  7


In [682]:
def check_missing_keys(df, c_dict):
    # Get value counts for each column in the df
    value_counts_dict = {col: df[col].value_counts().to_dict() for col in df.columns if col in c_dict}
    # To check later if the cols were filtered out
    missing_keys_columns = []
    # Checks for keys in the dataset that don't exist in the conversion dictionary
    for col, counts in value_counts_dict.items():
        conversion_keys = c_dict[col].keys()
        missing_keys = [key for key in counts.keys() if key not in conversion_keys and key != '' and key != float('nan')]
        if missing_keys:
            missing_keys_columns.append(col)
            print(f"Column '{col}' nominal values missing: {missing_keys}")
    return missing_keys_columns

In [683]:
cols_with_missing_keys = check_missing_keys(dataframe, conversion_dict)

"1 a 8 anos" and "9 a 11 anos" should be 6 or 7, but there's actually data that corresponds to a nominal value of 8, so "9 a 11 anos" will be considered being 8 for proximity, and its value will be the same as "8 a 11 anos" because of the superposing range

In [684]:
print(dataframe['ESC'][dataframe['ESC'] == '8'].value_counts())

ESC
8    1403
Name: count, dtype: int64


![image.png](../utils/infos/attributes-desc/imgs/ESC_bad_attrs.png)

In [685]:
conversion_dict['ESC']['8'] = '9 a 11 anos'

We need to standardize some attributes that hold similar values, like "ESC", "ESC2010" and "ESCFALAGR1"

![image.png](../utils/infos/attributes-desc/imgs/ESC%20table%20conversion.png)

In [686]:
# Attempt to convert the categorical data in scholarity (escolaridade) 
# attributes into a unified numerical data, because they represent 
# years of study, so technically there is a linear numerical correlation

# Standardizing the categorical values
ESC_MAP = {
    "0": 'Ignorado',        # Ignorado
    "1": 'Nenhuma',         # Nenhuma
    "2": '1 a 3 anos',      # 1 a 3 anos
    "3": '4 a 7 anos',      # 4 a 7 anos
    "4": '8 a 11 anos',     # 8 a 11 anos
    "5": '12 anos e mais',  # 12 anos e mais
    "8": '8 a 11 anos',     # 9 a 11 anos
    "9": 'Ignorado'         # Ignorado
}
ESC2010_MAP = {
    "0": 'Nenhuma',         # Sem escolaridade
    "1": '1 a 3 anos',      # Fundamental I (1ª a 4ª série)
    "2": '4 a 7 anos',      # Fundamental II (5ª a 8ª série)
    "3": '8 a 11 anos',     # Médio (antigo 2º Grau)
    "4": '12 anos e mais',  # Superior incompleto
    "5": '12 anos e mais',  # Superior completo
    "9": 'Ignorado'         # Ignorado
}
ESCFALAGR1_MAP = {
    "00": 'Nenhuma',        # Sem escolaridade
    "01": '1 a 3 anos',     # Fundamental I Incompleto
    "02": '4 a 7 anos',     # Fundamental I Completo
    "03": '4 a 7 anos',     # Fundamental II Incompleto
    "04": '8 a 11 anos',    # Fundamental II Completo
    "05": '8 a 11 anos',    # Ensino Médio Incompleto
    "06": '8 a 11 anos',    # Ensino Médio Completo
    "07": '12 anos e mais', # Superior Incompleto
    "08": '12 anos e mais', # Superior Completo
    "09": 'Ignorado',       # Ignorado
    "10": '1 a 3 anos',     # Fundamental I Incompleto ou Inespecífico
    "11": '4 a 7 anos',     # Fundamental II Incompleto ou Inespecífico
    "12": '8 a 11 anos'     # Ensino Médio Incompleto ou Inespecífico
}

conversion_dict['ESC'] = ESC_MAP
conversion_dict['ESC2010'] = ESC2010_MAP
conversion_dict['ESCFALAGR1'] = ESCFALAGR1_MAP


# Legacy conversion of categorical to numerical data of scholarity related attributes
#  # Transforming the data into numerical, a range of years is defined by the mean
# ESC_mean_MAP = {
#     "0": 0,   # Ignorado
#     "1": 0,   # Nenhuma
#     "2": 2,   # 1 a 3 anos
#     "3": 5.5, # 4 a 7 anos
#     "4": 9.5, # 8 a 11 anos
#     "5": 12,  # 12 anos e mais
#     "8": 9.5, # 9 a 11 anos
#     "9": 0    # Ignorado
# }
# ESC2010_mean_MAP = {
#     "0": 0,   # Sem escolaridade
#     "1": 2,   # Fundamental I (1ª a 4ª série)
#     "2": 5.5, # Fundamental II (5ª a 8ª série)
#     "3": 9.5, # Médio (antigo 2º Grau)
#     "4": 12,  # Superior incompleto
#     "5": 14,  # Superior completo
#     "9": 0    # Ignorado
# }
# ESCFALAGR1_mean_MAP = {
#     "00": 0,   # Sem escolaridade
#     "01": 2,   # Fundamental I Incompleto
#     "02": 4,   # Fundamental I Completo
#     "03": 5.5, # Fundamental II Incompleto
#     "04": 8,   # Fundamental II Completo
#     "05": 9.5, # Ensino Médio Incompleto
#     "06": 11,  # Ensino Médio Completo
#     "07": 12,  # Superior Incompleto
#     "08": 14,  # Superior Completo
#     "09": 0,   # Ignorado
#     "10": 2,   # Fundamental I Incompleto ou Inespecífico
#     "11": 5.5, # Fundamental II Incompleto ou Inespecífico
#     "12": 9.5  # Ensino Médio Incompleto ou Inespecífico

# Legacy conversion of categorical to numerical data of scholarity related attributes
# conversion_dict['ESC_mean'] = ESC_mean_MAP
# conversion_dict['ESC2010_mean'] = ESC2010_mean_MAP
# conversion_dict['ESCFALAGR1_mean'] = ESCFALAGR1_mean_MAP

RACACOR has a value "9" that doesn't exist according to the documents, so it will be counted as "ignored"

In [687]:
print(dataframe['RACACOR'][dataframe['RACACOR'] == '9'].value_counts())
conversion_dict['RACACOR']['9'] = 'Ignorado'

RACACOR
9    1
Name: count, dtype: int64


LOCOCOR has a value "0" that doesn't exist according to the documents, so it will be counted as "ignored"

In [688]:
print(dataframe['LOCOCOR'][dataframe['LOCOCOR'] == '0'].value_counts())
conversion_dict['LOCOCOR']['0'] = 'ignorado'

LOCOCOR
0    3
Name: count, dtype: int64


In [689]:
cols_with_missing_keys = check_missing_keys(dataframe, conversion_dict)
display(cols_with_missing_keys)

[]

----
# Handling the attributes

In [690]:
dir_to_attributes_info_table = '../brsuicides_info_tables'

file_preprocessing_types = dir_to_attributes_info_table + '/attributes_preprocessing_type.csv'
preprocessing_types = pd.read_csv(file_preprocessing_types)
display(preprocessing_types)

,Attribute,Conversion Type,Null value,Remove Missing/Invalid?
0,DTOBITO,ignore,NaN,True
1,ESTADO,ignore,NaN,True
2,ANO,ignore,NaN,True
3,CAUSABAS,ignore,NaN,True
4,SEXO,categorical,ignorado,False
5,RACACOR,categorical,ignorado,False
6,ESTCIV,categorical,ignorado,False
7,ESC,categorical,ignorado,False
8,ESC2010,categorical,ignorado,False
9,ESCFALAGR1,categorical,ignorado,False


In [691]:
# columns_that_dont_need_handling = ['ESTADO', 'ANO', 'DTOBITO', 'CAUSABAS']

# # no need because conversion_dict will take care of that
nominal_columns = list(set(list(dataframe.columns)).intersection(set(list(conversion_dict.keys()))))
# nominal_columns = ['SEXO', 'RACACOR', 'ESTCIV', 'ESC', 'ESC2010', 'ESCFALAGR1' 'LOCOCOR']

date_columns = ['DTNASC']
age_columns = ['IDADE']
ocupation_columns = ['OCUP']
city_codes_columns = ['CODMUNRES', 'CODMUNOCOR']
numeric_columns = ['NATURAL']

columns_with_special_types = {
    'DTNASC': 'date',
    'IDADE': 'age',
    'OCUP': 'ocupation',
    'CODMUNRES': 'city_codes',
    'CODMUNOCOR': 'city_codes',
    'NATURAL': 'numeric'
}

In [692]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267976 entries, 0 to 267975
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   ESTADO      267976 non-null  object        
 1   ANO         267976 non-null  int64         
 2   DTOBITO     267976 non-null  datetime64[ns]
 3   NATURAL     222906 non-null  object        
 4   DTNASC      266011 non-null  object        
 5   IDADE       267817 non-null  object        
 6   SEXO        267972 non-null  object        
 7   RACACOR     246441 non-null  object        
 8   ESTCIV      259609 non-null  object        
 9   ESC         226836 non-null  object        
 10  OCUP        217938 non-null  object        
 11  CODMUNRES   267976 non-null  int64         
 12  LOCOCOR     267572 non-null  object        
 13  CODMUNOCOR  267976 non-null  int64         
 14  CAUSABAS    267976 non-null  object        
 15  ESC2010     128028 non-null  object        
 16  ES

In [693]:
display(dataframe)

,ESTADO,ANO,DTOBITO,NATURAL,DTNASC,IDADE,SEXO,RACACOR,ESTCIV,ESC,OCUP,CODMUNRES,LOCOCOR,CODMUNOCOR,CAUSABAS,ESC2010,ESCFALAGR1
0,PR,1996,1996-10-24,841,5081963,433,1,NaN,1,1,61200,4119251,5,4119251,X709,NaN,NaN
1,PR,1996,1996-02-04,152,21091920,475,1,NaN,2,1,62100,4101903,3,4101903,X680,NaN,NaN
2,PR,1996,1996-01-17,841,28081961,434,1,NaN,1,NaN,62100,4100103,1,4100103,X680,NaN,NaN
3,PR,1996,1996-02-07,841,24091977,418,1,NaN,1,1,62100,4123204,1,4126009,X680,NaN,NaN
4,PR,1996,1996-02-09,841,30081956,439,1,NaN,5,NaN,62100,4123907,3,4123907,X700,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267971,MT,2022,2022-09-17,851,28081982,440,1,4,5,2,622005,510395,3,510395,X730,1,01
267972,MT,2022,2022-08-21,815,11091998,423,1,4,1,4,715210,510340,5,510340,X708,2,04
267973,MT,2022,2022-10-14,851,23061993,429,1,1,1,4,914405,510760,3,510760,X700,3,06
267974,MT,2022,2022-10-09,851,12061981,441,1,4,1,5,354705,510760,3,510760,X700,4,07


In [694]:
from preprocessing import preprocess_date_column, clean_categorical_values_column, clean_age_values_column
from preprocessing import clean_numeric_columns, clean_ocupation_column, clean_municipality_code_column

def clean_column_attribute_values(df, columns_special_types_dict, dir_to_info_table, cat_conversion_dict={}):
    values_types_dict = {key: [] for key in list(columns_special_types_dict.keys())}
    # Loop through the column value types dictionary
    for col, value_type in columns_special_types_dict.items():
        print(f'----------------------------------------------------')
        print(f'Cleaning {col} (doesn\'t remove data)...')
        # Check the value_type and apply the corresponding handling function
        if value_type == 'date':
            df, col_values_types = preprocess_date_column(df, col, remove_invalid=False)
        elif value_type == 'age':
            df, col_values_types = clean_age_values_column(df, col, 'DTNASC', 'DTOBITO')
        elif value_type == 'ocupation':
            df, col_values_types = clean_ocupation_column(df, col)
        elif value_type == 'city_codes':
            df, col_values_types = clean_municipality_code_column(df, col)
        elif value_type == 'numeric':
            df, col_values_types = clean_numeric_columns(df, col)
        if col_values_types:
            append_to_attributes_values_table(dir_to_info_table, col_values_types)

    # Loop through columns that exist in both df and cat_conversion_dict to convert categorical values
    for col, conversion_dict in cat_conversion_dict.items():
        if col in df.columns:
            df, col_values_types = clean_categorical_values_column(df, col, cat_conversion_dict[col])
            if col_values_types:
                append_to_attributes_values_table(dir_to_info_table, col_values_types)
        
    return df

In [695]:
cleaned_df = clean_column_attribute_values(dataframe, columns_with_special_types, dir_to_attributes_info_table, conversion_dict)

----------------------------------------------------
Cleaning DTNASC (doesn't remove data)...
Column: DTNASC
Unstandardized date example:
0     5081963
1    21091920
Name: DTNASC, dtype: object

Standardized date example:
0   1963-08-05
1   1920-09-21
Name: DTNASC, dtype: datetime64[ns]

Nº of not removed DTNASC invalid rows: 2122

----------------------------------------------------
Cleaning IDADE (doesn't remove data)...
Handling IDADE column...
Nº of age missing values: 157
Nº of age invalid values (age < 5): 611
Nº of age valid values (age >= 5): 267208
----------------------------------------------------
Cleaning OCUP (doesn't remove data)...
Handling OCUP column...
{10105: 'OFICIAL GENERAL DA AERONAUTICA', 10110: 'OFICIAL GENERAL DO EXERCITO', 10115: 'OFICIAL GENERAL DA MARINHA', 10205: 'OFICIAL DA AERONAUTICA', 10210: 'OFICIAL DO EXERCITO', 10215: 'OFICIAL DA MARINHA', 10305: 'PRACA DA AERONAUTICA', 10310: 'PRACA DO EXERCITO', 10315: 'PRACA DA MARINHA', 20105: 'CORONEL DA POLICI

In [696]:
display(cleaned_df)
display(cleaned_df.info())

,ESTADO,ANO,DTOBITO,NATURAL,DTNASC,IDADE,SEXO,RACACOR,ESTCIV,ESC,OCUP,CODMUNRES,LOCOCOR,CODMUNOCOR,CAUSABAS,ESC2010,ESCFALAGR1
0,PR,1996,1996-10-24,841.0,1963-08-05,33.0,masculino,NaN,Solteiro,Nenhuma,61200,Pinhal de São Bento,outros,Pinhal de São Bento,X709,NaN,NaN
1,PR,1996,1996-02-04,152.0,1920-09-21,75.0,masculino,NaN,Casado,Nenhuma,62100,Assaí,domicílio,Assaí,X680,NaN,NaN
2,PR,1996,1996-01-17,841.0,1961-08-28,34.0,masculino,NaN,Solteiro,NaN,62100,Abatiá,hospital,Abatiá,X680,NaN,NaN
3,PR,1996,1996-02-07,841.0,1977-09-24,18.0,masculino,NaN,Solteiro,Nenhuma,62100,Santa Cecília do Pavão,hospital,São Sebastião da Amoreira,X680,NaN,NaN
4,PR,1996,1996-02-09,841.0,1956-08-30,39.0,masculino,NaN,União consensual,NaN,62100,Santa Mariana,domicílio,Santa Mariana,X700,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267971,MT,2022,2022-09-17,851.0,1982-08-28,40.0,masculino,Parda,União consensual,1 a 3 anos,CASEIRO (AGRICULTURA),Glória D'Oeste,domicílio,Glória D'Oeste,X730,1 a 3 anos,1 a 3 anos
267972,MT,2022,2022-08-21,815.0,1998-09-11,23.0,masculino,Parda,Solteiro,8 a 11 anos,PEDREIRO,Cuiabá,outros,Cuiabá,X708,4 a 7 anos,8 a 11 anos
267973,MT,2022,2022-10-14,851.0,1993-06-23,29.0,masculino,Branca,Solteiro,8 a 11 anos,MECANICO DE MANUTENCAO DE AUTOMOVEIS E MOTOCIC...,Rondonópolis,domicílio,Rondonópolis,X700,8 a 11 anos,8 a 11 anos
267974,MT,2022,2022-10-09,851.0,1981-06-12,41.0,masculino,Parda,Solteiro,12 anos e mais,REPRESENTANTE COMERCIAL AUTONOMO,Rondonópolis,domicílio,Rondonópolis,X700,12 anos e mais,12 anos e mais


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267976 entries, 0 to 267975
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   ESTADO      267976 non-null  object        
 1   ANO         267976 non-null  int64         
 2   DTOBITO     267976 non-null  datetime64[ns]
 3   NATURAL     222906 non-null  float64       
 4   DTNASC      265854 non-null  datetime64[ns]
 5   IDADE       267819 non-null  float64       
 6   SEXO        267972 non-null  object        
 7   RACACOR     246441 non-null  object        
 8   ESTCIV      259609 non-null  object        
 9   ESC         226836 non-null  object        
 10  OCUP        217938 non-null  object        
 11  CODMUNRES   267976 non-null  object        
 12  LOCOCOR     267572 non-null  object        
 13  CODMUNOCOR  267976 non-null  object        
 14  CAUSABAS    267976 non-null  object        
 15  ESC2010     128028 non-null  object        
 16  ES

None

In [697]:
# This next step is responsible for handling the missing, null and invalid values.

attribute_removal_strategies = pd.DataFrame(preprocessing_types.set_index('Attribute'))
display(attribute_removal_strategies.transpose())

Attribute,DTOBITO,ESTADO,ANO,CAUSABAS,SEXO,RACACOR,ESTCIV,ESC,ESC2010,ESCFALAGR1,LOCOCOR,DTNASC,IDADE,OCUP,CODMUNRES,CODMUNOCOR,NATURAL
Conversion Type,ignore,ignore,ignore,ignore,categorical,categorical,categorical,categorical,categorical,categorical,categorical,date,age,ocupation,city_codes,city_codes,numeric
Null value,NaN,NaN,NaN,NaN,ignorado,ignorado,ignorado,ignorado,ignorado,ignorado,ignorado,NaN,1.919191,NaN,numeric,numeric,NaN
Remove Missing/Invalid?,True,True,True,True,False,False,False,False,False,False,False,False,True,False,True,True,False


In [698]:
def finish_column_attribute_values_preprocessing(df, removal_strategy, dir_to_info_table):
    invalid_values_columns_dict = {}

    # Loop through each column in the removal strategy
    for attribute, strategy in removal_strategy.iterrows():
        null_value = strategy['Null value']
        remove_invalid = strategy['Remove Missing/Invalid?']
        conversion_type = strategy['Conversion Type']

        if not remove_invalid or conversion_type == 'ignore':
            continue

        if attribute in df.columns:
            invalid_rows = []

            if null_value == 'numeric':
                # Convert to numeric and identify invalid values (text that couldn't convert)
                valid_numeric = pd.to_numeric(df[attribute], errors='coerce')
                # filters by numeric values (codes that didnt become text) and values that are NaN in df
                invalid_rows = list(df.index[valid_numeric.notna() | df[attribute].isna()])
            else:
                invalid_rows = df.index[df[attribute].astype(str).str.lower() == str(null_value).lower()].union(df.index[df[attribute].isna()])
            invalid_values_columns_dict[attribute] = invalid_rows

    # Flatten indexes to remove duplicates
    all_invalid_indexes = set([idx for invalid in invalid_values_columns_dict.values() for idx in invalid])
    all_invalid_indexes_length = len(all_invalid_indexes)
    print(f'Number of unique indexes that contain invalid values: {all_invalid_indexes_length}')

    # Remove the invalid indexes from the DataFrame
    df = df.drop(index=all_invalid_indexes)

    for col in invalid_values_columns_dict:
        value_count = len(invalid_values_columns_dict[col])
        n_of_preprocessed = len(df) - value_count
        col_info_values = [col, float('nan'), float('nan'), float('nan'), n_of_preprocessed, value_count]
        update_attribute_values(dir_to_info_table, col_info_values)

    for col in set(df.columns).difference(invalid_values_columns_dict.keys()):
        n_of_preprocessed = len(df)
        col_info_values = [col, float('nan'), float('nan'), float('nan'), n_of_preprocessed, float('nan')]
        update_attribute_values(dir_to_info_table, col_info_values)

    return df, invalid_values_columns_dict

In [699]:
preprocessed_df, removed_data_dict = finish_column_attribute_values_preprocessing(cleaned_df, attribute_removal_strategies, dir_to_attributes_info_table)

Number of unique indexes that contain invalid values: 1749
Updated values for 'IDADE' in the datatable.
Updated values for 'CODMUNRES' in the datatable.
Updated values for 'CODMUNOCOR' in the datatable.
Updated values for 'OCUP' in the datatable.
Updated values for 'CAUSABAS' in the datatable.
Updated values for 'ESCFALAGR1' in the datatable.
Updated values for 'NATURAL' in the datatable.
Updated values for 'RACACOR' in the datatable.
Updated values for 'ESC2010' in the datatable.
Updated values for 'DTNASC' in the datatable.
Updated values for 'DTOBITO' in the datatable.
Updated values for 'ANO' in the datatable.
Updated values for 'ESC' in the datatable.
Updated values for 'ESTADO' in the datatable.
Updated values for 'ESTCIV' in the datatable.
Updated values for 'LOCOCOR' in the datatable.
Updated values for 'SEXO' in the datatable.


## Creating the csv

In [700]:
preprocessed_df.to_csv(csv_data_dir_final + final_dataset_name + '-' + str(years[0]) + '_' + str(years[-1]) + '.csv', index=False)

In [701]:
pd.read_csv(csv_data_dir_final + final_dataset_name + '-' + str(years[0]) + '_' + str(years[-1]) + '.csv')

,ESTADO,ANO,DTOBITO,NATURAL,DTNASC,IDADE,SEXO,RACACOR,ESTCIV,ESC,OCUP,CODMUNRES,LOCOCOR,CODMUNOCOR,CAUSABAS,ESC2010,ESCFALAGR1
0,PR,1996,1996-10-24,841.0,1963-08-05,33.0,masculino,NaN,Solteiro,Nenhuma,61200.0,Pinhal de São Bento,outros,Pinhal de São Bento,X709,NaN,NaN
1,PR,1996,1996-02-04,152.0,1920-09-21,75.0,masculino,NaN,Casado,Nenhuma,62100.0,Assaí,domicílio,Assaí,X680,NaN,NaN
2,PR,1996,1996-01-17,841.0,1961-08-28,34.0,masculino,NaN,Solteiro,NaN,62100.0,Abatiá,hospital,Abatiá,X680,NaN,NaN
3,PR,1996,1996-02-07,841.0,1977-09-24,18.0,masculino,NaN,Solteiro,Nenhuma,62100.0,Santa Cecília do Pavão,hospital,São Sebastião da Amoreira,X680,NaN,NaN
4,PR,1996,1996-02-09,841.0,1956-08-30,39.0,masculino,NaN,União consensual,NaN,62100.0,Santa Mariana,domicílio,Santa Mariana,X700,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266222,MT,2022,2022-09-17,851.0,1982-08-28,40.0,masculino,Parda,União consensual,1 a 3 anos,CASEIRO (AGRICULTURA),Glória D'Oeste,domicílio,Glória D'Oeste,X730,1 a 3 anos,1 a 3 anos
266223,MT,2022,2022-08-21,815.0,1998-09-11,23.0,masculino,Parda,Solteiro,8 a 11 anos,PEDREIRO,Cuiabá,outros,Cuiabá,X708,4 a 7 anos,8 a 11 anos
266224,MT,2022,2022-10-14,851.0,1993-06-23,29.0,masculino,Branca,Solteiro,8 a 11 anos,MECANICO DE MANUTENCAO DE AUTOMOVEIS E MOTOCIC...,Rondonópolis,domicílio,Rondonópolis,X700,8 a 11 anos,8 a 11 anos
266225,MT,2022,2022-10-09,851.0,1981-06-12,41.0,masculino,Parda,Solteiro,12 anos e mais,REPRESENTANTE COMERCIAL AUTONOMO,Rondonópolis,domicílio,Rondonópolis,X700,12 anos e mais,12 anos e mais
